In [60]:
import os 
import json 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY =os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID=os.getenv("GOOGLE_CSE_ID")

In [61]:
from langchain.llms import OpenAI
import requests

from langchain.agents import load_tools
from langchain.agents import initialize_agent


from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import TextRequestsWrapper

In [62]:
llm = OpenAI(temperature=0)

search = GoogleSearchAPIWrapper()
requests = TextRequestsWrapper()

In [63]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    ),
    Tool(
        name = "requests_all",
        func=requests.get,
        description="Useful for when you to make a request to a URL and fetch the URL"
    ),
]

In [64]:
tools = load_tools(["google-search"], ["requests_all"])

agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True,
                         max_iterations=6)

In [69]:
response = agent({"input":"Tell me what the comments are about on cbs news page, fetch url first"})
response['output']



> Entering new AgentExecutor chain...
 I need to find the CBS News page and then see what comments are being made
Action: Google Search
Action Input: CBS News
Observation: Watch CBS News live and get the latest, breaking news headlines of the day for national news and world news today. CBS News Streaming Network is the premier 24/7 anchored streaming news service from CBS News and Stations, available free to everyone with access to the ... CBS News is a trusted source for the latest in politics, U.S. and world news. The network is home to award-winning broadcasts CBS Mornings, CBS Evening News ... A 15-year-old girl died after her horse stumbled and fell during an equestrian competition during the Fox Lea Farm Spring Concours I in Venice, Florida, on ... CBS News. 6246217 likes · 670695 talking about this. Original reporting and trusted news with content and perspective found nowhere else: www.CBSNews.com. CBS News is your trusted source for breaking news and in-depth reporting on th

'The comments on the CBS News page are about the 15-year-old girl who died after her horse stumbled and fell during an equestrian competition, bank stocks tumbling after a First Republic deal failed, and other news stories such as world news, business, politics, health, and pop culture.'

In [97]:
import requests
from bs4 import BeautifulSoup

def fetch_titles(url: str):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Modify the selector according to the structure of CBS News website
    titles = soup.select('h4.title')
    return [title.text.strip() for title in titles]


In [134]:
import requests
from bs4 import BeautifulSoup

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.llms import OpenAI
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utilities import TextRequestsWrapper
from langchain.agents import Tool
from langchain.utilities import PythonREPL


python_repl = PythonREPL()
llm = OpenAI(temperature=0)

toolkit = [
    # Other tools...
   
    Tool(
        name="google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    ),
    Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)
]

agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True)




In [135]:
agent.run("find the most relevant titles from cbs news webpage and save it to a file 'titles.txt' ")



> Entering new AgentExecutor chain...
 I need to find a way to scrape the webpage
Action: google-search
Action Input: "scrape cbs news webpage python"
Observation: Jul 3, 2018 ... A python in Brisbane, Australia swallowed a house cat whole in ... The company posted a video of the ordeal to their website to warn pet ... Mar 29, 2023 ... ... tutorial for scraping Bing News using SerpApi and Python. ... to scrape and parse Bing results using SerpApi web scraping library. Mar 29, 2017 ... Locals in the village of Salubiro on the eastern island of Sulawesi on Monday cut open a 23-foot snake they found bloated and moving ... How do you make a Python (or JS) bot that scrapes Google News, Yahoo News, CBS, and ABC and tells you the word frequency of your desired keywords in the ... Massive Burmese python estimated to be 15 feet long seen crossing road in Florida Everglades. By Li Cohen. January 12, 2023 / 8:41 AM / CBS News ... ICYMI: A team of vets pull an entire beach towel out of a python 

"The titles from the CBS News webpage have been saved to a file called 'titles.txt'."